In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
import os
import time
import numpy as np
client_id = "72c9f2b9d3344c91af50a24f4144228f"
client_secret = "249943a9bbb74b9a8bf04bb7fd93790d"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # sp is the instance of the spotipy api


In [2]:
data = pd.read_csv('/Users/ytkd/Desktop/LY_Artist_lyrics_genre_data_from_big5_mft_users_likes_final.csv')

data = data[data['lang_detect_spacy']=='en'] # Selecting the data which has the lang_detect_spacy column as "en"
dataset_artists = list(data['Artist']) # Creating a list of all the artists from our dataset
dataset_titles = list(data['title']) # Creating a list of all the titles from our dataset
data.iloc[18]

Artist                                                    1 Giant Leap
title                                                     Braided Hair
original_lyrics      [Intro Speech]\nYeah? Yo we was talkin earlier...
hasLyrics                                                          Yes
isOrchestral                                                        No
first_genre_tag                                                 No-tag
second_genre_tag                                                No-tag
cleaned_lyrics       Yeah? Yo we was talkin earlier and I was -I wa...
lang_detect_spacy                                                   en
Name: 19, dtype: object

In [3]:
'''
Creating a list which has both artists and their songs
'''
            
artist_title_dataset = []
for i in range(len(data)):
    artist_title_dataset.append(str(data['Artist'].iloc[i]) + ' ' + str(data['title'].iloc[i]))
artist_title_dataset[0]

'Nsync Bye Bye Bye'

In [4]:
records_ascii = []
records_foreign_char= []

def is_ascii(s):
  return s.isascii()

for item in artist_title_dataset:
    if is_ascii(item) is True:
        records_ascii.append(item)
    else:
        records_foreign_char.append(item)

In [5]:
records_ascii[0]

'Nsync Bye Bye Bye'

In [6]:
print('The total number of records with only ascii :- ',len(records_ascii))
print('Whereas, the total number of records with foreign characters are :- ',len(records_foreign_char))

The total number of records with only ascii :-  40598
Whereas, the total number of records with foreign characters are :-  5640


<h3>In the below cell I will remove all the non ascii characters from the 5640 records and query the resulting string to spotify's api. Spotify will understand the name of the artist even if there are one or two characters misplaced or missing. Hence we will use this to clean our artist names. We will replace our faulty artist names with the artist anmes that spotify gives when we make a query using the records without the foreign characters.

The below regular expression will remove all the non ascii characters from our string

In [7]:
import re

def only_ascii(item):
    rex = re.sub(r'[^\x00-\x7F]+',' ', item)
    return rex

In [8]:
artist_ascii = []
title_ascii = []
for artist in dataset_artists:
     artist_ascii.append(only_ascii(str(artist)))

for title in dataset_titles:
     title_ascii.append(only_ascii(str(title)))


In [9]:
''' Creating a combined list of artists and titles'''
ascii_artist_title = []
for item in range(len(artist_ascii)):
    ascii_artist_title.append(artist_ascii[item] + ' ' + title_ascii[item])

In [10]:
'''
In track_preview_available function I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available.

In is_same_artist function I check whether the artist the we found in our query is the same one from our dataset.
'''
def is_same_artist(query, data, index):
    if data[index] in query['tracks']['items'][0]['artists'][0]['name']:
        return True

def track_preview_available(query):
    if query['tracks']['items']:
        if query['tracks']['items'][0]['preview_url']:
            return True
        return True
        
def artist_available(query):
    if query['tracks']['items']:
        return True


<h3> Correcting artist names

In [11]:
print('Total number of tracks',len(artist_ascii))
print('Unique artists are: ',len(set(artist_ascii)))

unique_artists_ascii = list(set(artist_ascii))
unique_artists_ascii[0]

Total number of tracks 46238
Unique artists are:  5461


'Camille'

<h4>Querying all the artists and correcting their names using the real name of spotify artists

In [26]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()
artist_from_spotify = []
artist_names_changed = []
artist_not_found = []

for index in range(len(artist_ascii[:50])):
    artist_name = artist_ascii[index]
    song_title = title_ascii[index]
    search = f'{ascii_artist_title[index]}'
    query = sp.search(search, type = 'track')
    
    if artist_available(query) and is_same_artist(query,artist_ascii,index):
        artist_from_spotify.append(query['tracks']['items'][0]['artists'][0]['name'])
        artist_names_changed.append(query['tracks']['items'][0]['artists'][0]['name'])

    else:
        artist_from_spotify.append(artist_ascii[index])
        artist_not_found.append(artist_ascii[index])
        
    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " requests sent")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 requests sent
Loop #: 5
Elapsed Time: 2.012716054916382 seconds
10 requests sent
Loop #: 10
Elapsed Time: 4.973083257675171 seconds
15 requests sent
Loop #: 15
Elapsed Time: 7.352055072784424 seconds
20 requests sent
Loop #: 20
Elapsed Time: 10.437409162521362 seconds
25 requests sent
Loop #: 25
Elapsed Time: 12.844887256622314 seconds
30 requests sent
Loop #: 30
Elapsed Time: 15.809081077575684 seconds
35 requests sent
Loop #: 35
Elapsed Time: 19.356504201889038 seconds
40 requests sent
Loop #: 40
Elapsed Time: 22.42341923713684 seconds
45 requests sent
Loop #: 45
Elapsed Time: 25.185692310333252 seconds
50 requests sent
Loop #: 50
Elapsed Time: 27.225393295288086 seconds


In [30]:
artist_names_changed

['12 Stones', '12 Stones', '12 Stones', '12 Stones']

<h3> Now we have cleaned artist names, and hence we can now use these names and extract the track's preview_urls, track_ids, artist_ids, popularity and other faetures

In [29]:
# print(f'Out of {len(artist_ascii[:301])} artists that we considered we cleaned {len(artist_names_changed)} artist names')
print(f'We considered {len(artist_from_spotify)} sample records and were able to clean {len(artist_names_changed)}')

We considered 50 sample records and were able to clean 4


In [71]:
sleep_min = 1
sleep_max = 3
request_count = 0
start_time = time.time()

track_data = []

for index in range(len(artist_from_spotify)):
    artist_name = artist_ascii[index]
    song_title = title_ascii[index]
    search = f'artist:{artist_name} track:{song_title}'
    query = sp.search(search, type='track')


    # in the below line of code I am checking that the query returned by spotify's api is not empty and if its not I check if the preview url is available, and
    # even further I check a third condition whether the artist the we found in our query is the same from our dataset.


    # if query['tracks']['items'] and query['tracks']['items'][0]['preview_url'] and query['tracks']['items'][0]['artists'][0]['name'] in alpha_artists_titles[index]:
    if track_preview_available(query) and is_same_artist(query,ascii_artist_title,index):
        artist_from_spotify.append(query['tracks']['items'][0]['artists'][0]['name'])

    request_count+=1
    if request_count % 5 == 0:
        print(str(request_count) + " artists uris fetched")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        print('Loop #: {}'.format(request_count))
        print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 artists uris fetched
Loop #: 5
Elapsed Time: 1.8791790008544922 seconds
10 artists uris fetched
Loop #: 10
Elapsed Time: 5.853464126586914 seconds
15 artists uris fetched
Loop #: 15
Elapsed Time: 9.114161252975464 seconds
20 artists uris fetched
Loop #: 20
Elapsed Time: 11.25255036354065 seconds
25 artists uris fetched
Loop #: 25
Elapsed Time: 13.402014970779419 seconds
30 artists uris fetched
Loop #: 30
Elapsed Time: 16.36712908744812 seconds
35 artists uris fetched
Loop #: 35
Elapsed Time: 18.885977268218994 seconds
40 artists uris fetched
Loop #: 40
Elapsed Time: 22.002456188201904 seconds
45 artists uris fetched
Loop #: 45
Elapsed Time: 25.801711320877075 seconds
50 artists uris fetched
Loop #: 50
Elapsed Time: 27.914245128631592 seconds


29